# Realni primer

<a href="https://archive.ics.uci.edu/ml/datasets/iris"><b>Iris Data Set</b></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("data/iris.data", header=None, names=["sepal-length", "sepal-width", "petal-length", "petal-width", "class"])

colors = {"Iris-setosa": 0,
         "Iris-versicolor": 1,
         "Iris-virginica": 2}

for key, value in colors.items():
    filter_ = data["class"] == key
    data.loc[filter_, "class"] = value
data["class"] = pd.to_numeric(data["class"]) # before, pandas took them as strings. But we need them as integers for our AI model

print(data.info())
print(data.head())

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 12))
axs[0][0].scatter(data["sepal-length"], data["sepal-width"], c=data["class"])
axs[0][0].set_title("sepal-length VS sepal-width")

axs[0][1].scatter(data["sepal-length"], data["petal-length"], c=data["class"])
axs[0][1].set_title("sepal-length VS petal-length")

axs[0][2].scatter(data["sepal-length"], data["petal-width"], c=data["class"])
axs[0][2].set_title("sepal-length VS petal-width")

axs[1][0].scatter(data["sepal-width"], data["petal-length"], c=data["class"])
axs[1][0].set_title("sepal-width VS petal-length")

axs[1][1].scatter(data["sepal-width"], data["petal-width"], c=data["class"])
axs[1][1].set_title("sepal-width VS petal-width")

axs[1][2].scatter(data["petal-length"], data["petal-width"], c=data["class"])
axs[1][2].set_title("petal-length VS petal-width")

plt.show()

---

In [ ]:
from sklearn.model_selection import train_test_split

print(data.head())

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=420)
print(X_train[:5])
print(y_train[:5])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(2020)

class Layer_Dense:

    # Layer initialization
    def __init__(self, n_inputs, n_neurons,
                         weight_regularizer_l1=0, 
                         weight_regularizer_l2=0,
                         bias_regularizer_l1=0, 
                         bias_regularizer_l2=0):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        # Set regularization strength
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

        # Gradients on regularization
        # L1 on weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
        # L2 on weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
        # L1 on biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
        # L2 on biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)



# ReLU activation
class Activation_ReLU:

    # Forward pass
    def forward(self, inputs):

        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Since we need to modify original variable,
        # let's make a copy of values first
        self.dinputs = dvalues.copy()

        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0


# Softmax activation
class Activation_Softmax:

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs

        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1,
                                            keepdims=True))
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1,
                                            keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):

        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in \
                enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output and
            jacobian_matrix = np.diagflat(single_output) - \
                              np.dot(single_output, single_output.T)

            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix,
                                         single_dvalues)


# Common loss class
class Loss:
    
    # Regularization loss calculation
    def regularization_loss(self, layer):

        # 0 by default
        regularization_loss = 0

        # L1 regularization - weights
        # calculate only when factor greater than 0
        if layer.weight_regularizer_l1 > 0:
            regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))

        # L2 regularization - weights
        if layer.weight_regularizer_l2 > 0:
            regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)


        # L1 regularization - biases
        # calculate only when factor greater than 0
        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

        # L2 regularization - biases
        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)

        return regularization_loss

    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):

        # Calculate sample losses
        sample_losses = self.forward(output, y)

        # Calculate mean loss
        data_loss = np.mean(sample_losses)

        # Return loss
        return data_loss


# Cross-entropy loss
class Loss_CategoricalCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):

        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]


        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples


# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy():

    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)


    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)

        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)

        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples


        
# SGD optimizer
class Optimizer_SGD:

    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):
        layer.weights += -self.current_learning_rate * layer.dweights
        layer.biases += -self.current_learning_rate * layer.dbiases

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

# <=== HERE ===>
# Create dataset
X = X_train
y = y_train

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(4, 16)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(16, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(learning_rate=0.005, decay=1e-3)
# <=== HERE ===>

# Train in loop
losses = [] # Used to plot loss values and see how our model learned
for epoch in range(10_001):

    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    data_loss = loss_activation.forward(dense2.output, y)

    # Calculate regularization penalty
    regularization_loss = \
        loss_activation.loss.regularization_loss(dense1) + \
        loss_activation.loss.regularization_loss(dense2)

    # Calculate overall loss
    loss = data_loss + regularization_loss

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        losses.append(loss)
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

    
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 24))
axs[0].scatter(X[:, 0], X[:, 1], c=y, cmap="brg", marker="o", s=500, alpha=0.6)
ax2 = axs[0].twinx()
ax2.scatter(X[:, 0], X[:, 1], c=predictions, cmap="brg", marker="o", s=100, edgecolors="black")

axs[1].plot(range(len(losses)), losses)
axs[1].set_xlabel("100 * Epoch")
axs[1].set_ylabel("Loss")
plt.show()

In [ ]:
# <=== HERE ===>
# We have already devided dataset into training and testing
# <=== HERE ===>

# Perform a forward pass of our training data through this layer
dense1.forward(X_test)

# Perform a forward pass through activation function
# takes the output of first dense layer here
activation1.forward(dense1.output)

# Perform a forward pass through second Dense layer
# takes outputs of activation function of first layer as inputs
dense2.forward(activation1.output)


# Perform a forward pass through the activation/loss function
# takes the output of second dense layer here and returns loss
data_loss = loss_activation.forward(dense2.output, y_test)

# Calculate regularization penalty
regularization_loss = \
    loss_activation.loss.regularization_loss(dense1) + \
    loss_activation.loss.regularization_loss(dense2)

# Calculate overall loss
loss = data_loss + regularization_loss

# Calculate accuracy from output of activation2 and targets
# calculate values along first axis
predictions = np.argmax(loss_activation.output, axis=1)
if len(y_test.shape) == 2:
    y_test = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions==y_test)

print(f'acc: {accuracy:.3f}, loss: {loss:.3f}')

fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap="brg", marker="o", s=500, alpha=0.6)
ax2 = ax.twinx()
ax2.scatter(X_test[:, 0], X_test[:, 1], c=predictions, cmap="brg", marker="o", s=100, edgecolors="black")
plt.show()

---

# Keras, Tensorflow

In [ ]:
data = pd.read_csv("data/iris.data", header=None, names=["sepal-length", "sepal-width", "petal-length", "petal-width", "class"])
print(data.info())

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

print(X.head())
print(y.head())

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

print(y)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
print(encoded_Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=420)
print(X_train[:5])
print(y_train[:5])

---

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

## Creating Neural Network Model


In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(16, input_dim=4, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(learning_rate=0.005),
              metrics=["accuracy"])

In [ ]:
model.summary()

## Training

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 12))
ax.plot(history.history["loss"])
plt.show()

## Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f} \t Accuracy: {accuracy*100:.2f}%")

## Making predictions

In [ ]:
predictions = model.predict(X_test)
print(predictions[:5])

In [ ]:
for prediction in predictions[:5]:
    print(f"SOFTMAX output:\n{prediction}\n")

    i = np.argmax(prediction)
    print(f"Index največje vrednosti je {i}\n")
    
    flower_class = encoder.inverse_transform([i])
    print(f"Napoved: \t {flower_class}")
    print(30*"=")

In [ ]:
predictions = model.predict(X_test)


fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=np.argmax(y_test, axis=1), cmap="brg", marker="o", s=500, alpha=0.6)
ax2 = ax.twinx()
ax2.scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=np.argmax(y_test, axis=1), cmap="brg", marker="o", s=100, edgecolors="black")
plt.show()

# Save model

In [ ]:
model.save("models/iris.h5")

In [ ]:
from tensorflow.keras.models import load_model
model2 = load_model("models/iris.h5")

loss, accuracy = model2.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f} \t Accuracy: {accuracy*100:.2f}%")

predictions = model2.predict(X_test)

fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=np.argmax(y_test, axis=1), cmap="brg", marker="o", s=500, alpha=0.6)
ax2 = ax.twinx()
ax2.scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=np.argmax(y_test, axis=1), cmap="brg", marker="o", s=100, edgecolors="black")
plt.show()

# Vaja

Ustvarite Dense model in ga uporabite na *Prima Indians onset of diabetes* datasetu. Naredite klasifikacijo ali je oseba dobila diabetes v roku 5 let ali ne.

## Load data

*Prima Indians onset of diabetes dataset* opisuje zdravstvene podatke pacientov Prima Indians porekla in napoveduje ali so imeli diabetes v roku 5 let ali ne.

Dataset je namenjen binarni klasifikaciji:
* **1** - So dobili diabetes
* **0** - Niso dobili diabetes



In [ ]:
import pandas as pd

In [ ]:
columns = [
    "Number of times pregnant",
    "Plasma glucose concentration a 2 hours in an oral glucose tolerance test",
    "Diastolic blood pressure (mm Hg)",
    "Triceps skin fold thickness (mm)",
    "2-Hour serum insulin (mu U/ml)",
    "Body mass index (weight in kg/(height in m)^2)",
    "Diabetes pedigree function",
    "Age (years)",
    "Class variable"
]
data = pd.read_csv("data/pima-indians-diabetes.data.csv", names=columns)
data.head()

---

## Rešitev